# Assignment 6 Report
# Name - Ambuj Mishra
# Student ID - 202116003

##### This is a colab PDF containing both question codes and the description required in the assignment.

# Spark Initialization

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
import findspark

findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
type(spark)

pyspark.sql.session.SparkSession

In [ ]:
sc=spark.sparkContext

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Question-1(a)

## From Table 3-2

In [ ]:
#####  creating a RDD to process further
nums = sc.parallelize([1, 2, 3, 4])

### Using map()

In [ ]:
#map() 
#Apply a function to each element in the RDD and return an RDD of the result.
squared = nums.map(lambda  x : x**2)
squared.collect()

[1, 4, 9, 16]

### Using flatMap()

In [ ]:
nums.flatMap(lambda x : x.to(3))

PythonRDD[2] at RDD at PythonRDD.scala:53

### Using filter()

In [ ]:
nums.filter(lambda x : x == 1).collect()

[1]

### Using Distinct()

In [ ]:
nums.distinct().collect()

[1, 2, 3, 4]

### Using sample()

In [ ]:
nums.sample(False, 0.5,1).collect()

[1, 3]

## From Table 3-3

In [ ]:
nums1 = sc.parallelize([1, 2, 3])
nums2 = sc.parallelize([3, 4, 5])

### Using union()

In [ ]:
nums1.union(nums2).collect()

[1, 2, 3, 3, 4, 5]

### Using intersection()

In [ ]:
nums1.intersection(nums2).collect()

[3]

### Using subtract()

In [ ]:
nums1.subtract(nums2).collect()

[2, 1]

### Using cartesian()

In [ ]:
nums1.cartesian(nums2).collect()

[(1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 3), (3, 4), (3, 5)]

## From Table 3-4

### Using collect()

In [ ]:
basicRDD = sc.parallelize([1, 2, 3, 3])

In [ ]:
basicRDD.collect()

[1, 2, 3, 3]

### Using count()

In [ ]:
basicRDD.count()

4

### Using countByValue()

In [ ]:
basicRDD.countByValue()

defaultdict(int, {1: 1, 2: 1, 3: 2})

### Using take()

In [ ]:
basicRDD.take(3)

[1, 2, 3]

### Using top()

In [ ]:
basicRDD.top(2)

[3, 3]

### Using takeOrdered()

In [ ]:
basicRDD.takeOrdered(3)

[1, 2, 3]

### Using takeSample()

In [ ]:
basicRDD.takeSample(False, 2)

[3, 1]

### Using reduce()

In [ ]:
basicRDD.reduce(lambda x, y: x+y)

9

### Using fold()

In [ ]:
basicRDD.fold(0, lambda x, y :  x+y) 

9

### Using aggregate()

In [ ]:
print(basicRDD.aggregate((0, 0) , (lambda  x, y :  (x[0] + y, x[1] + 1)),  (lambda  x, y : (x[0] + y[0], x[1] + y[1]))))

(9, 4)


### Using foreach()

In [ ]:
def my_print(x):
  print(x+1)

basicRDD.foreach(my_print)

# Question-1(b)

In [ ]:
from pyspark.storagelevel import StorageLevel

result = basicRDD.map(lambda x : x * x)
result.persist(StorageLevel.DISK_ONLY)

print(result.collect())
print(result.count())

[1, 4, 9, 9]
4


# Question-2

## Implementing Page Rank algorithm on Spark-shell

In [ ]:
##### The screenshot regarding the page rank problem is added in the next page.

# Question-3(a)

## Importing Dataset

In [ ]:
# in Python
sales = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/content/drive/MyDrive/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv").coalesce(5).where("Description IS NOT NULL")
fakeIntDF = spark.read.parquet("/content/drive/MyDrive/Spark-The-Definitive-Guide-master/data/simple-ml-integers")
simpleDF = spark.read.json("/content/drive/MyDrive/Spark-The-Definitive-Guide-master/data/simple-ml")
scaleDF = spark.read.parquet("/content/drive/MyDrive/Spark-The-Definitive-Guide-master/data/simple-ml-scaling")

In [ ]:
sales.cache()
sales.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

## A.) Using StandardScaler

In [ ]:
from pyspark.ml.feature import StandardScaler

sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+-----------------------------------+
| id|      features|StandardScaler_e63c4ca325e1__output|
+---+--------------+-----------------------------------+
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  1|[3.0,10.1,3.0]|               [3.58568582800318...|
+---+--------------+-----------------------------------+



## B.) Using MinMaxScaler

In [ ]:
from pyspark.ml.feature import MinMaxScaler

minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

+---+--------------+---------------------------------+
| id|      features|MinMaxScaler_7bc11441f185__output|
+---+--------------+---------------------------------+
|  0|[1.0,0.1,-1.0]|                    [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                    [7.5,5.5,7.5]|
|  0|[1.0,0.1,-1.0]|                    [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                    [7.5,5.5,7.5]|
|  1|[3.0,10.1,3.0]|                 [10.0,10.0,10.0]|
+---+--------------+---------------------------------+



## C.) MaxAbsScaler

In [ ]:
from pyspark.ml.feature import MaxAbsScaler

maScaler = MaxAbsScaler().setInputCol("features")
fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show()

+---+--------------+---------------------------------+
| id|      features|MaxAbsScaler_756c5bb55422__output|
+---+--------------+---------------------------------+
|  0|[1.0,0.1,-1.0]|             [0.33333333333333...|
|  1| [2.0,1.1,1.0]|             [0.66666666666666...|
|  0|[1.0,0.1,-1.0]|             [0.33333333333333...|
|  1| [2.0,1.1,1.0]|             [0.66666666666666...|
|  1|[3.0,10.1,3.0]|                    [1.0,1.0,1.0]|
+---+--------------+---------------------------------+



## D.) Elementwise-Product

In [ ]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors

scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
scalingUp = ElementwiseProduct().setScalingVec(scaleUpVec).setInputCol("features")
scalingUp.transform(scaleDF).show()

+---+--------------+---------------------------------------+
| id|      features|ElementwiseProduct_46832125aaa3__output|
+---+--------------+---------------------------------------+
|  0|[1.0,0.1,-1.0]|                       [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]|                       [20.0,16.5,20.0]|
|  0|[1.0,0.1,-1.0]|                       [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]|                       [20.0,16.5,20.0]|
|  1|[3.0,10.1,3.0]|                      [30.0,151.5,60.0]|
+---+--------------+---------------------------------------+



## E.) Normalizer

In [ ]:
from pyspark.ml.feature import Normalizer

manhattanDistance = Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show()

+---+--------------+-------------------------------+
| id|      features|Normalizer_8d0ba3515a2a__output|
+---+--------------+-------------------------------+
|  0|[1.0,0.1,-1.0]|           [0.47619047619047...|
|  1| [2.0,1.1,1.0]|           [0.48780487804878...|
|  0|[1.0,0.1,-1.0]|           [0.47619047619047...|
|  1| [2.0,1.1,1.0]|           [0.48780487804878...|
|  1|[3.0,10.1,3.0]|           [0.18633540372670...|
+---+--------------+-------------------------------+



# Question-3(b)

## A.) Tokenizing Text Before Processing

In [ ]:
from pyspark.ml.feature import Tokenizer

tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(20, False)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, , water, transfer, tattoos]      |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

In [ ]:
from pyspark.ml.feature import RegexTokenizer

rt = RegexTokenizer().setInputCol("Description").setOutputCol("DescOut").setPattern(" ").setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, water, transfer, tattoos]        |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

In [ ]:
# in Python
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescOut")\
.setPattern(" ")\
.setGaps(False)\
.setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------+
|Description                        |DescOut           |
+-----------------------------------+------------------+
|RABBIT NIGHT LIGHT                 |[ ,  ]            |
|DOUGHNUT LIP GLOSS                 |[ ,  ,  ]         |
|12 MESSAGE CARDS WITH ENVELOPES    |[ ,  ,  ,  ]      |
|BLUE HARMONICA IN BOX              |[ ,  ,  ,  ]      |
|GUMBALL COAT RACK                  |[ ,  ]            |
|SKULLS  WATER TRANSFER TATTOOS     |[ ,  ,  ,  ,  ]   |
|FELTCRAFT GIRL AMELIE KIT          |[ ,  ,  ]         |
|CAMOUFLAGE LED TORCH               |[ ,  ]            |
|WHITE SKULL HOT WATER BOTTLE       |[ ,  ,  ,  ,  ]   |
|ENGLISH ROSE HOT WATER BOTTLE      |[ ,  ,  ,  ]      |
|HOT WATER BOTTLE KEEP CALM         |[ ,  ,  ,  ]      |
|SCOTTIE DOG HOT WATER BOTTLE       |[ ,  ,  ,  ]      |
|ROSE CARAVAN DOORSTOP              |[ ,  ]            |
|GINGHAM HEART  DOORSTOP RED        |[ ,  ,  ,  ]      |
|STORAGE TIN VINTAGE LEAF      

## B.) Removing Common Words

In [ ]:
from pyspark.ml.feature import StopWordsRemover

englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover().setStopWords(englishStopWords).setInputCol("DescOut")
stops.transform(tokenized).show()

+--------------------+--------------------+-------------------------------------+
|         Description|             DescOut|StopWordsRemover_2ff782587456__output|
+--------------------+--------------------+-------------------------------------+
|  RABBIT NIGHT LIGHT|[rabbit, night, l...|                 [rabbit, night, l...|
| DOUGHNUT LIP GLOSS |[doughnut, lip, g...|                 [doughnut, lip, g...|
|12 MESSAGE CARDS ...|[12, message, car...|                 [12, message, car...|
|BLUE HARMONICA IN...|[blue, harmonica,...|                 [blue, harmonica,...|
|   GUMBALL COAT RACK|[gumball, coat, r...|                 [gumball, coat, r...|
|SKULLS  WATER TRA...|[skulls, , water,...|                 [skulls, , water,...|
|FELTCRAFT GIRL AM...|[feltcraft, girl,...|                 [feltcraft, girl,...|
|CAMOUFLAGE LED TORCH|[camouflage, led,...|                 [camouflage, led,...|
|WHITE SKULL HOT W...|[white, skull, ho...|                 [white, skull, ho...|
|ENGLISH ROSE HO

## C.) Creating Word Combinations

In [ ]:
from pyspark.ml.feature import NGram

unigram = NGram().setInputCol("DescOut").setN(1)
bigram = NGram().setInputCol("DescOut").setN(2)

In [ ]:
unigramRDD = unigram.transform(tokenized.select("DescOut"))
unigramRDD.show()

+--------------------+--------------------------+
|             DescOut|NGram_6679e9d34f1a__output|
+--------------------+--------------------------+
|[rabbit, night, l...|      [rabbit, night, l...|
|[doughnut, lip, g...|      [doughnut, lip, g...|
|[12, message, car...|      [12, message, car...|
|[blue, harmonica,...|      [blue, harmonica,...|
|[gumball, coat, r...|      [gumball, coat, r...|
|[skulls, , water,...|      [skulls, , water,...|
|[feltcraft, girl,...|      [feltcraft, girl,...|
|[camouflage, led,...|      [camouflage, led,...|
|[white, skull, ho...|      [white, skull, ho...|
|[english, rose, h...|      [english, rose, h...|
|[hot, water, bott...|      [hot, water, bott...|
|[scottie, dog, ho...|      [scottie, dog, ho...|
|[rose, caravan, d...|      [rose, caravan, d...|
|[gingham, heart, ...|      [gingham, heart, ...|
|[storage, tin, vi...|      [storage, tin, vi...|
|[set, of, 4, knic...|      [set, of, 4, knic...|
|   [popcorn, holder]|         [popcorn, holder]|


In [ ]:
bigramRDD = bigram.transform(tokenized.select("DescOut"))
bigramRDD.show()

+--------------------+--------------------------+
|             DescOut|NGram_96fe20ee8337__output|
+--------------------+--------------------------+
|[rabbit, night, l...|      [rabbit night, ni...|
|[doughnut, lip, g...|      [doughnut lip, li...|
|[12, message, car...|      [12 message, mess...|
|[blue, harmonica,...|      [blue harmonica, ...|
|[gumball, coat, r...|      [gumball coat, co...|
|[skulls, , water,...|      [skulls ,  water,...|
|[feltcraft, girl,...|      [feltcraft girl, ...|
|[camouflage, led,...|      [camouflage led, ...|
|[white, skull, ho...|      [white skull, sku...|
|[english, rose, h...|      [english rose, ro...|
|[hot, water, bott...|      [hot water, water...|
|[scottie, dog, ho...|      [scottie dog, dog...|
|[rose, caravan, d...|      [rose caravan, ca...|
|[gingham, heart, ...|      [gingham heart, h...|
|[storage, tin, vi...|      [storage tin, tin...|
|[set, of, 4, knic...|      [set of, of 4, 4 ...|
|   [popcorn, holder]|          [popcorn holder]|


# THANK YOU!

In [ ]:
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('Big_Data06.ipynb')

File ‘colab_pdf.py’ already there; not retrieving.





[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/Big_Data06.ipynb to pdf
[NbConvertApp] Writing 75281 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 75443 bytes to /content/drive/My Drive/Big_Data06.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'